In [23]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [114]:
url_inicial = 'https://www.probdes.iiec.unam.mx/index.php/pde/issue/archive/9'
url_root = 'https://www.probdes.iiec.unam.mx/index.php/pde/issue/archive/9'
r=requests.get(url_inicial)

In [115]:
soup = BeautifulSoup(r.text, 'html.parser')

In [116]:
box = soup.find('div', class_='issues media-list')
volumen=box.findAll('div', class_='media-body')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['https://www.probdes.iiec.unam.mx/index.php/pde/issue/view/3613',
 'https://www.probdes.iiec.unam.mx/index.php/pde/issue/view/3592',
 'https://www.probdes.iiec.unam.mx/index.php/pde/issue/view/3591']

In [117]:
def get_url_items(sopa, url): 
    box = soup.find('div', class_='issues media-list')
    volumen=box.findAll('div', class_='media-body')
    vol = [x.find('a').get('href')for x in volumen]
    vol=[urljoin(url_root,i) for i in vol]
    return vol

In [118]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links)

Estoy en la pagina https://www.probdes.iiec.unam.mx/index.php/pde/issue/archive/9


In [119]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

3

In [120]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [121]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Problemas del Desarrollo. Revista Latinoamericana de Economía'
    a='V. Ciencias Sociales'
    tem='Economía, Desarrollo Económico'
    url='http://www.probdes.iiec.unam.mx/'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo de revista']=tire
    except AttributeError:
        content_book['Titulo de revista']=None
    #area
    try:
        area=a
        content_book['Área']=a
    except AttributeError:
        content_book['Área']=None
    #tematica
    try:
        tema=tem
        content_book['Temática']=tem
    except AttributeError:
        content_book['Temática']=None
    #titulo libro
    try:
        titu=s_item.find('h3', class_='media-heading').get_text(strip=True)
        content_book['Titulo del libro']=titu
    except AttributeError:
        content_book['Titulo del libro']=None
    #resumen
    try:
        resu=s_item.find('a', class_='cover').get_text(strip=True)
        content_book['Resumen']=resu
    except AttributeError:
        content_book['Resumen']=None
    #Link incial
    try:
        inicial=url
        content_book['Link inicial']=url
    except AttributeError:
        content_book['Link inicial']=None
    #link articulo
    try:
        link=s_item.find('a', class_='galley-link btn btn-primary pdf').get('href')
        content_book['Link articulo']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link articulo']=None
    return content_book

In [122]:
list_scraper=list_scraper[0:3]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2


In [123]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo de revista,Área,Temática,Titulo del libro,Resumen,Link inicial,Link articulo
0,Problemas del Desarrollo. Revista Latinoameric...,V. Ciencias Sociales,"Economía, Desarrollo Económico",A nuestros lectores,,http://www.probdes.iiec.unam.mx/,https://www.probdes.iiec.unam.mx/index.php/pde...
1,Problemas del Desarrollo. Revista Latinoameric...,V. Ciencias Sociales,"Economía, Desarrollo Económico",A nuestros lectores,,http://www.probdes.iiec.unam.mx/,https://www.probdes.iiec.unam.mx/index.php/pde...
2,Problemas del Desarrollo. Revista Latinoameric...,V. Ciencias Sociales,"Economía, Desarrollo Económico",A nuestros lectores,,http://www.probdes.iiec.unam.mx/,https://www.probdes.iiec.unam.mx/index.php/pde...


In [124]:
df_catalogo.to_csv('Revista69.09.csv', index=False)